In [1]:
import os
os.environ['YT_DEST'] = '/home/kwoksun2/anaconda2/pkgs/yt-3.3.5-np111py27_2/'
import numpy as np
from dengo.chemical_network import \
    ChemicalNetwork, \
    reaction_registry, \
    cooling_registry, species_registry
import dengo.primordial_rates, dengo.primordial_cooling
from dengo.chemistry_constants import tiny, kboltz, mh
import yt
import numpy
from IPython.display import Image
import matplotlib.pyplot as plt
import pickle
import pyximport
pyximport.install(setup_args={"include_dirs":np.get_include()},
                      reload_support=True, inplace=True)
loc_pwd = "/home/kwoksun2/grackle/src/python/examples/"
def save_obj(obj, name ):
    with open(loc_pwd+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(loc_pwd + name + '.pkl', 'rb') as f:
        return pickle.load(f)

/home/kwoksun2/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


 using cli
 using CLI for selections


In [3]:
# only two species


def Init_values(temperature, density, fH, reaction_k):
    
    X = 1.0
    # initialize and setup the network
    dengo.primordial_rates.setup_primordial()
    primordial = ChemicalNetwork()

    for i in range(23):
        if (i+1) not in [0]: #9, 10, 11, 17,18,19]:
            try:
                primordial.add_reaction("k{0:02d}".format(i+1))
            except:
                pass
    
    primordial.init_temperature((1e0, 1e5))

    tiny = 1.0e-20

    # init_array are is in fractional abundances
    init_array = density
    init_values = dict()
    
    init_values["He_1"]    = density * (1.0 - 0.76) /3.
    init_values["He_2"]    = density * (1.0 - 0.76) /3.      
    init_values["He_3"]    = np.array([tiny])         
    init_values["H_1"]     = density *  (0.76)  /3.
    init_values['H_2']     = density *  (0.76)  /3.
    
    init_values["H_m0"]    = np.array([tiny]) #density *  (0.76)  /4.
    init_values["H2_1"]    = density *  (0.76)  /3.
    init_values["H2_2"]    = np.array([tiny]) # density *  (0.76)  /5.
    
    
#     init_values["He_1"]    = density * (1.0 - 0.76) 
#     init_values["He_2"]    = np.array([tiny]) #density * (1.0 - 0.76) /3.      
#     init_values["He_3"]    = np.array([tiny]) #density * (1.0 - 0.76) /3.          
#     init_values["H_1"]     = density *  (0.76)  
#     init_values['H_2']     = np.array([tiny]) #density *  (0.76)  /4.
    
#     init_values["H_m0"]    = np.array([tiny]) #density *  (0.76)  /4.
#     init_values["H2_1"]    = np.array([tiny]) #density *  (0.76)  /4.
#     init_values["H2_2"]    = np.array([tiny]) # density *  (0.76)  /5.    

    # now everything in mass density
#    init_values = primordial.convert_to_mass_density(init_values)
    init_values['de'] = primordial.calculate_free_electrons(init_values)
    # one signle value: again mass density
    init_values['density'] = primordial.calculate_total_density(init_values)
    number_density = primordial.calculate_number_density(init_values)
    
    print(init_values)
    print('density: {}'.format(init_values['density']))
    print('number density: {}'.format(number_density))
    # set up initial temperatures values used to define ge
    init_values['T'] = temperature

    # calculate ge (very crudely, no H2 help here)
    gamma = 5.0/3.0
    init_values['ge'] = ((temperature * number_density * kboltz)
                         / (init_values['density'] * mh * (gamma - 1)))
    return init_values, primordial

In [4]:
def main(fH, temp, den , full_output = False):
    
    
    temp = temp.flatten()
    den = den.flatten()
    
    h_array = numpy.zeros((len(temp), len(den)))
    t_array = numpy.zeros((len(den)))
    
    h_array = []
    t_array = []
    
    print(den)
    for iden, den_arr in enumerate(den):
        
        den_arr = numpy.ones((len(temp)))*den_arr
        
        init_values, primordial = Init_values(temp, den_arr, fH)
        h_array_now, t_array_now = equil_values(init_values, primordial, full_output = full_output)
        
        h_array.append(h_array_now)
        t_array.append(t_array_now)
        
    return h_array, t_array

In [5]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

def make_plots( network, init, rv_int, solver_name = 'sundials', t_min = 1e0, t_max = 1e11, plot_fH = False ):
    
    rv_arr = rv_int
    
    mask = rv_arr['successful']
    t    = rv_arr['t'][mask]
    dt   = rv_arr['dt'][mask]

    plt.clf()
    skip_list = ['dt' ,'successful', 't','Ju', 'Jac', 'ge']
    hydrogen_list = ['H_1','H_2','H2_1','H2_2','H_m0']
    
    
    # Calculate total electron density 
    # with the abundances from other species
    total_e = numpy.zeros((len(t)))
    for j in network.required_species:
        if j.name in rv_arr.keys() and j.name not in skip_list and j.name != 'dt':
            
            try:
                total_e += rv_arr[j.name][0][mask]  * j.free_electrons 
            except:
                pass
    
    de = rv_arr['de'][0][mask]
    # calculate total mass density of hydrogen
    # conservation of mass
    total_Hatoms = numpy.zeros((len(t)))
    for j in network.required_species:
        if j.name in hydrogen_list:
            total_Hatoms += rv_arr[j.name][0][mask] *j.weight
            

    f, axarr = plt.subplots(3, sharex=True, figsize=(10,15))
    
    ii = 0
    for i in sorted(rv_arr.keys()):
        if i not in skip_list:
            
            hi = rv_arr[i][0][mask]
            if t_min == 1e0:
                axarr[0].semilogx(t,numpy.array(hi)/init[i][0]+ii, label=i) 
                ii+=1
            else:
                idx = find_nearest(t , t_min)
                axarr[0].semilogx(t[idx:] ,numpy.array(hi[idx:])/hi[idx] , label=i) 
                ii+=1



    #h1 = rv_arr['H_1'][0][mask]

    
    T0 = rv_arr['T'][0][mask][0]
    axarr[0].set_title('{}:  Temperature: {} K     Density: {} per cc '.format(solver_name, T0 , total_Hatoms[0]   ))
    axarr[0].set_xlabel('Time (s)')
    axarr[0].set_ylabel('Species abundances (normalized by initial values)')
    
    dt = rv_arr['dt'][mask]
 #   if t_min == 1e0:
 #       axarr[0].loglog(t, dt / dt[0], label=i) 
 #   else:
 #       idx = find_nearest(t , t_min)
 #       axarr[0].loglog(t[idx:] ,dt[idx:] / dt[idx], label=i) 
                
    

    de = rv_arr['de'][0][mask]
    axarr[0].loglog(t, total_e/de, label='de normalized by those from species', ls = '--', color = 'r')

    axarr[0].legend(loc='best',fontsize='x-small')
    # plt.savefig(con_not_con+'-species-norm-nHe20.png')


    for i in sorted(rv_arr.keys()):
        if i not in skip_list:
            hi = rv_arr[i][0][mask]
            axarr[1].loglog(t,hi, label=i) 


    axarr[1].set_xlabel('Time (s)')
    axarr[1].set_ylabel('Species abundances')

    axarr[1].legend(loc='best',fontsize='x-small')


    

    de = rv_arr['de'][0][mask]
    axarr[2].semilogx(t, (de/total_e) , label='de normalized by those from species', ls = '--', color = 'r')

    axarr[2].semilogx(t, ( total_Hatoms )/ total_Hatoms[0], label = 'mass density of H atoms' )
    
    
    print('de conserved?   ', de[-1]/total_e[-1])
    print('mass conserved? ' , ( total_Hatoms[-1] )/ total_Hatoms[0])
    
    
    if plot_fH:
        axarr[2].plot(t, h1/total_Hatoms, label="f(H)" )
        print('final f(H) = {}'.format(h1[-1]/total_Hatoms[-1]) )
    axarr[2].set_xlabel('Time (s)')
    axarr[2].set_ylabel('mass ratio of Hydrogen atom (fH)')
    axarr[2].legend()
    
    axarr[2].set_xlim((t_min, t_max))
    
    T0 = rv_arr['T'][0][0]
    f.savefig(solver_name+'_nH={:.2E}_T={:.1E}.png'.format(total_Hatoms[0], T0) )
    plt.close(f)
    return solver_name+'_nH={:.2E}_T={:.1E}.png'.format(total_Hatoms[0], T0)

In [6]:

class my_network:
    def __init__(self, T):
        self.T = T

from sympy import *

def equi_fH(rho, T, k):
    
    
    init, network = Init_values(T,rho, 0.0, k)
    
    # for i in network.required_species:
    #     print(network.print_ccode(i))
    #     for j in network.required_species:
    #         print(network.print_jacobian_component(i,j))

    mH = 1.0

    k13 = (network.reactions['k13'])
    k22 = (network.reactions['k22'])

    temp_network = my_network(T)

    k13coeff, k22coeff = (k13.coeff_fn(temp_network), k22.coeff_fn(temp_network))
    

    k13s, k22s, fHs, rhos = symbols(" k13 k22 fH rho")
    expr = k13s*(1 - fHs) /2 - k22s*fHs*fHs*rhos
    known_vals = [ (k13s, k13coeff), (k22s, k22coeff), (rhos, rho) ]
    new_expr = expr.subs(known_vals)
    new_expr_poly = Poly(new_expr, fHs)
    print('Temperature now: {} K'.format(T))
    poly_coeff = (new_expr_poly.coeffs())
    ans = numpy.roots(poly_coeff)
    print('fH: {}'.format(ans[ans>0]) )
    return ans[ans>0]

Temp_array = numpy.array([3.0e3])
# in atomic mass unit! not grams
rho = numpy.array([1.0e15])

In [7]:
%%time
ith_T = numpy.array([1000.0])
rho = numpy.array([1.00e10 ])
    

init, primordial = Init_values( ith_T , rho, 0, 0 )

primordial.write_solver("sunny", output_dir = ".",
                            init_values=init,
                            input_is_number=False)

sunny_solver_run = pyximport.load_module("sunny_solver_run",
                                "sunny_solver_run.pyx",
                                build_inplace = True, pyxbuild_dir = "_dengo_temp")


rv, rv_int_be_chem = sunny_solver_run.run_sunny(init, 1e5 , niter=2e5 )

Adding reaction: k01 : 1*H_1 + 1*de => 1*H_2 + 2*de
Adding reaction: k02 : 1*H_2 + 1*de => 1*H_1
Adding reaction: k03 : 1*He_1 + 1*de => 1*He_2 + 2*de
Adding reaction: k04 : 1*He_2 + 1*de => 1*He_1
Adding reaction: k05 : 1*He_2 + 1*de => 1*He_3 + 2*de
Adding reaction: k06 : 1*He_3 + 1*de => 1*He_2
Adding reaction: k07 : 1*H_1 + 1*de => 1*H_m0
Adding reaction: k08 : 1*H_m0 + 1*H_1 => 1*H2_1 + 1*de
Adding reaction: k09 : 1*H_1 + 1*H_2 => 1*H2_2
Adding reaction: k10 : 1*H2_2 + 1*H_1 => 1*H2_1 + 1*H_2
Adding reaction: k11 : 1*H2_1 + 1*H_2 => 1*H2_2 + 1*H_1
Adding reaction: k12 : 1*H2_1 + 1*de => 2*H_1 + 1*de
Adding reaction: k13 : 1*H2_1 + 1*H_1 => 3*H_1
Adding reaction: k14 : 1*H_m0 + 1*de => 1*H_1 + 2*de
Adding reaction: k15 : 1*H_m0 + 1*H_1 => 2*H_1 + 1*de
Adding reaction: k16 : 1*H_m0 + 1*H_2 => 2*H_1
Adding reaction: k17 : 1*H_m0 + 1*H_2 => 1*H2_2 + 1*de
Adding reaction: k18 : 1*H2_2 + 1*de => 2*H_1
Adding reaction: k19 : 1*H2_2 + 1*H_m0 => 1*H_1 + 1*H2_1
Adding reaction: k21 : 2*H_1 

ImportError: Building module sunny_solver_run failed: ["CompileError: command 'gcc' failed with exit status 1\n"]

In [ ]:
mask = rv_int_be_chem['successful']
t_arr = rv_int_be_chem['t'][mask]

png_name = make_plots( primordial, init, rv_int_be_chem, solver_name = 'be_chem_solve', t_min = t_arr[0], t_max = t_arr[-1]  )
print(png_name)
Image(png_name)

In [12]:
%%time
ith_T = numpy.array([1000.0])
rho = numpy.array([1.0e10])

init, primordial = Init_values( ith_T , rho, 0, 0 )

primordial.write_solver("sundials_time", solver_template = "sundials_time", 
                    ode_solver_source = "cvodes_solver_time.C", 
                    output_dir = ".",
                        init_values=init,
                        input_is_number=False)

sundials_solver_run = pyximport.load_module("sundials_time_solver_run",
                            "sundials_time_solver_run.pyx",
                            build_inplace = True, pyxbuild_dir = "_dengo_temp")
rv, rv_int_sundials = sundials_solver_run.run_sundials_time(init, 1.0e5 , niter=1e5) # , dtarr = dt_arr)

Adding reaction: k01 : 1*H_1 + 1*de => 1*H_2 + 2*de
Adding reaction: k02 : 1*H_2 + 1*de => 1*H_1
Adding reaction: k03 : 1*He_1 + 1*de => 1*He_2 + 2*de
Adding reaction: k04 : 1*He_2 + 1*de => 1*He_1
Adding reaction: k05 : 1*He_2 + 1*de => 1*He_3 + 2*de
Adding reaction: k06 : 1*He_3 + 1*de => 1*He_2
Adding reaction: k07 : 1*H_1 + 1*de => 1*H_m0
Adding reaction: k08 : 1*H_m0 + 1*H_1 => 1*H2_1 + 1*de
Adding reaction: k09 : 1*H_1 + 1*H_2 => 1*H2_2
Adding reaction: k10 : 1*H2_2 + 1*H_1 => 1*H2_1 + 1*H_2
Adding reaction: k11 : 1*H2_1 + 1*H_2 => 1*H2_2 + 1*H_1
Adding reaction: k12 : 1*H2_1 + 1*de => 2*H_1 + 1*de
Adding reaction: k13 : 1*H2_1 + 1*H_1 => 3*H_1
Adding reaction: k14 : 1*H_m0 + 1*de => 1*H_1 + 2*de
Adding reaction: k15 : 1*H_m0 + 1*H_1 => 2*H_1 + 1*de
Adding reaction: k16 : 1*H_m0 + 1*H_2 => 2*H_1
Adding reaction: k17 : 1*H_m0 + 1*H_2 => 1*H2_2 + 1*de
Adding reaction: k18 : 1*H2_2 + 1*de => 2*H_1
Adding reaction: k19 : 1*H2_2 + 1*H_m0 => 1*H_1 + 1*H2_1
Adding reaction: k21 : 2*H_1 

KeyboardInterrupt: 

In [ ]:
mask = rv_int_sundials['successful']
t_arr = rv_int_sundials['t'][mask]
png_name = make_plots( primordial, init, rv_int_sundials, solver_name = 'ttsundials', t_min = t_arr[0], t_max = t_arr[-1])
Image(png_name)



In [ ]:


mask = rv_int_be_chem['successful']
be_chem_data = {}

mask_s = rv_int_sundials['successful']
sundials_data = {}


#for name in ['H_1','H_2','He_1','He_2','He_3', 'de', 't']:
for name in ['H_1','H_2','He_1','He_2','He_3', 'H_m0', 'H2_1', 'H2_2', 'de', 't']:
#for name in ['H_1','H_2','He_1','He_2','He_3', 'H_m0', 'H2_1', 'de', 't']:
#for name in ['He_1','He_2','He_3', 'de', 't']:
    if name != 't':
        arr = rv_int_be_chem[name][0][mask]
        be_chem_data[name] = arr
        
        arr = rv_int_sundials[name][0][mask_s]
        sundials_data[name] = arr        
        
    else:
        arr = rv_int_be_chem[name][mask]
        be_chem_data[name] = arr
        
        arr = rv_int_sundials[name][mask_s]
        sundials_data[name] = arr
        
save_obj(be_chem_data, 'be_chem_data')
save_obj(sundials_data, 'sundials_data')

t_arr = rv_int_be_chem['t'][mask]
numpy.save("/home/kwoksun2/grackle/src/python/examples/t_arr.npy", t_arr)

In [ ]:
from timeit import default_timer as timer

start = timer()
rho_arr = np.logspace(0,20,21)
T_arr   = np.logspace(2,5,21)

xx, yy = numpy.meshgrid(rho_arr, T_arr)
rho_T_pair = zip(xx.flatten(),yy.flatten())
end = timer()
print(end-start)

In [13]:
for ith_T, rho in rho_T_pair:
    start = timer()
    init, primordial = Init_values( np.array([ith_T]) , np.array([rho]), 0, 0 )

    primordial.write_solver("sundials_time", solver_template = "sundials_time", 
                        ode_solver_source = "cvodes_solver_time.C", 
                        output_dir = ".",
                            init_values=init,
                            input_is_number=False)

    sundials_solver_run = pyximport.load_module("sundials_time_solver_run",
                                "sundials_time_solver_run.pyx",
                                build_inplace = True, pyxbuild_dir = "_dengo_temp")
    rv, rv_int_sundials = sundials_solver_run.run_sundials_time(init, 1.0e8 , niter=1e5) # , dtarr = dt_arr)
    end = timer()
    print('***************************')
    print(ith_T, rho)
    print('time taken: {0:.3f}'.format(end-start))

NameError: name 'rho_T_pair' is not defined

In [9]:
primordial.add_cooling("ceHI")

{'ceHI': ceHI_ceHI[i]}
{}


NameError: name 'HI' is not defined

In [ ]:
species_registry

In [ ]:
cool_glover = cooling_registry['gloverabel08']

In [ ]:
cool_glover.equation